In [2]:

import nltk 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize, sent_tokenize 
import os
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
stop_words = set(stopwords.words('english')) 
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
from tensorflow.keras.layers import concatenate,CuDNNLSTM
from tensorflow.keras.layers import Dense, LSTM,Dropout, Activation,Bidirectional

In [3]:
'''
Breaking whole data into list of sentences
'''
def seperate_sentences(data):
    return sent_tokenize(data) 

In [4]:
def Sentence_to_POS(all_sentences):

    data=[]
    for sentence in all_sentences: 
        one_sentence_pos = []
        '''
        Word tokenizers is used to find the words and punctuation in a string 
        '''  
        wordsList = nltk.word_tokenize(sentence) 

        '''
        Removing stop words from wordList.
        ''' 
        wordsList = [w for w in wordsList if not w in stop_words]  

        '''
        Using a Tagger. Which is part-of-speech tagger or POS-tagger.  
        '''

        tagged = nltk.pos_tag(wordsList) 
        for x in tagged:
            one_sentence_pos.append(x[1])
        data.append(one_sentence_pos)

    return data

In [5]:
def Fianl_X_and_Y(novel_POS,novel_label,number_of_sentenes_in_one):
    '''
    This funtion takes generates more data for training by breaking one novel into multiple novels
    with k number of sentences (number_of_sentenes_in_one) .we are ignoring some last sentences if they
    dont fit
    '''
    i=0
    X=[]
    Y=[]
    for novel in novel_POS:
        start = 0
        end = number_of_sentenes_in_one
        while(end<=len(novel)):
            X.append(novel[start:end])
            Y.append(novel_label[i])
            start=end
            end=end+number_of_sentenes_in_one
        i+=1
    return X,Y

In [6]:
def Get_X_and_Y_In_POS_Form(path_of_dataset_directory,number_of_sentences_in_one):
    '''
    this function takes the path of directory where the dataset is and returns the processed X and Y.
    The function calls will let u understand more the flow of code
    '''
    all_files = os.listdir(path_of_dataset_directory)
    novel_POS=[]
    novel_label=[]
    for filename in all_files:
        full_path = 'dataset/three_author_dataset/'+filename
        with open(full_path, 'r') as f:
            data = f.read().replace('"\n"','').replace('\n',' ').replace('- ','')
            all_sentences = seperate_sentences(data)
            sentences_to_pos = Sentence_to_POS(all_sentences)
            novel_POS.append(sentences_to_pos)
            novel_label.append(filename[8])
    return Fianl_X_and_Y(novel_POS,novel_label,number_of_sentences_in_one)


In [7]:
def tag_to_index_dictionary(X):
    '''
    This function just creates a dictionary from the POS tags which we
    encountered in our dataset.As the network works with numbers so simple will make
    dictionary which stores the index of associated POS tag
    '''
    tag = set([])
    for doc in X:
        for sentence in doc:
            for word in sentence:
                tag.add(word)
    tag2index = {t: i + 1 for i, t in enumerate(list(tag))}
    tag2index['-PAD-'] = 0
    return tag2index

In [8]:
def convert_tag_to_sequence_numbers(X):
    tag2index = tag_to_index_dictionary(X)
    '''
    using the tag2index dictionary assign indexs to the POS tags in our data
    '''
    new_X=[]
    for doc in X:
        new_S = []
        for sentence in doc:
            new_W=[]
            for word in sentence:
                new_W.append(tag2index[word])
            new_S.append(new_W)
        new_X.append(new_S)
    return new_X

In [9]:
def pad_zeros_to_sequence(X,max_length):
    '''
    This function padds zeros to the sequences so as to make fixed sequences
    According to paper 15 is best value for max_length
    '''
    
    new_X=[]
    for doc in X:
        new_X.append(pad_sequences(doc, maxlen=max_length, padding='post'))
    return new_X

In [10]:
def Encode_Labels(Y):
    '''
    This fucntion encodes the labels i.e., assign numbers to the labels
    '''
    le = LabelEncoder()
    return le.fit_transform(Y)

In [18]:
X,Y = Get_X_and_Y_In_POS_Form('dataset/three_author_dataset',100)

UnicodeDecodeError: 'ascii' codec can't decode byte 0xe2 in position 5: ordinal not in range(128)

In [17]:
full_path = 'dataset/three_author_dataset/12ItrainA1.TXT'
with open(full_path, 'r') as f:
    data = f.read().replace('"\n"','').replace('\n',' ').replace('- ','')
data.split('\n')

['\xe2\x80\x9cOh, yes, it is common knowledge,\xe2\x80\x9d said Travok Ott expansively, leaning back, sipping his light white wine with a most delicate air. \xe2\x80\x9cDelia, the Princess Majestrix, is continually indulging in affairs. Why, her latest inamorato is this muscular wrestler, Turko. Oh, yes, a lovely man. Who can blame her?\xe2\x80\x9d\r \r The perfumed currents of warmed air moved caressingly about the group of men sitting in the ord chamber of the Baths of the Nine. The chamber presented a comfortable, modish, relaxing atmosphere. Young girl slaves carried wine and parclear in glazed ceramic flagons, and bronze trays of sweetmeats and tempting cakes. No lady bathers were allowed here, their establishment was separated off by a stout masonry wall. The scented air cloyed.\r \r \xe2\x80\x9cSurely, this is just rumor, Travok?\xe2\x80\x9d said Urban the Gloves, popping a paline into his mouth.\r \r \xe2\x80\x9cHardly.\xe2\x80\x9d Travok Ott, a slender man with the brown hair 

In [ ]:
A,B,C=0,0,0
X_data=[]
for i in range(len(X)):
    if(Y[i]=='A' and A<95):
        X_data.append([X[i],Y[i]])
        A+=1
    elif(Y[i]=='B' and B<95):
        X_data.append([X[i],Y[i]])
        B+=1
    elif(Y[i]=='C' and C<95):
        X_data.append([X[i],Y[i]])
        C+=1


In [72]:
import random
random.shuffle(X_data)

In [73]:
new_X=[]
new_Y=[]
for m , n in X_data:
    new_X.append(m)
    new_Y.append(n)

In [74]:
np.unique(new_Y,return_counts=True)

(array(['A', 'B', 'C'], dtype='<U1'), array([95, 95, 95]))

In [75]:
X=new_X
Y=new_Y

In [76]:
DICTIONARY_LENGTH = len(tag_to_index_dictionary(X))
MAX_LENGTH = 30

In [77]:
X = convert_tag_to_sequence_numbers(X)

In [78]:
X = pad_zeros_to_sequence(X,MAX_LENGTH)

In [79]:
X = np.array(X)
X.shape

(285, 100, 30)

In [83]:
real_X = []
for i in range(100):
    real_X.append(list())
for i in range(100):
    for j in range(285):
        real_X[i].append(X[j][i])

In [85]:
X_train = []
for i in range(100):
    X_train.append(np.array(real_X[i]))

In [93]:
np.array(real_X).shape

(100, 285, 30)

In [80]:
Y = Encode_Labels(Y) 

In [90]:
from tensorflow.keras.utils import to_categorical
y_binary = to_categorical(Y)
y_binary.shape

(285, 3)

In [91]:
y_binary.shape

(285, 3)

In [140]:
X_t = []
for i in range(3):
    X_t.append(X_train[i])

(3, 285, 30)

In [153]:
outputs=[]
inputs_=[]
for i in range(3):
    inputlayer = Input(shape=[30])
    inputs_.append(inputlayer)
    layer = Embedding(DICTIONARY_LENGTH, 50, input_length=30)(inputlayer)
    layer = LSTM(50)(layer)
    layer = Dense(30)(layer)
    layer = Activation('relu')(layer)
    outputs.append(layer)
merge = concatenate(outputs)
hidden1 = Dense(10, activation='relu')(merge)
out = Dense(3, activation='softmax')(hidden1)
model = Model(inputs=inputs_,outputs=out)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           (None, 30)           0                                            
__________________________________________________________________________________________________
input_18 (InputLayer)           (None, 30)           0                                            
__________________________________________________________________________________________________
input_19 (InputLayer)           (None, 30)           0                                            
__________________________________________________________________________________________________
embedding_30 (Embedding)        (None, 30, 50)       2250        input_17[0][0]                   
__________________________________________________________________________________________________
embedding_

In [155]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Nadam(lr=0.001),
              metrics=['accuracy'])
model.fit(X_t,y_binary,epochs=2,
          validation_split=0.1)

Train on 256 samples, validate on 29 samples


InvalidArgumentError: No OpKernel was registered to support Op 'CudnnRNN' used by node cu_dnnlstm/CudnnRNN (defined at <ipython-input-144-3df2c9fd9452>:7) with these attrs: [seed=0, dropout=0, T=DT_FLOAT, input_mode="linear_input", direction="unidirectional", rnn_mode="lstm", is_training=true, seed2=0]
Registered devices: [CPU, XLA_CPU]
Registered kernels:
  <no registered kernels>

	 [[node cu_dnnlstm/CudnnRNN (defined at <ipython-input-144-3df2c9fd9452>:7) ]]

Caused by op 'cu_dnnlstm/CudnnRNN', defined at:
  File "/home/danish/anaconda3/lib/python3.7/runpy.py", line 193, in _run_module_as_main
  File "/home/danish/anaconda3/lib/python3.7/runpy.py", line 85, in _run_code
  File "/home/danish/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
  File "/home/danish/anaconda3/lib/python3.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
  File "/home/danish/anaconda3/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 505